**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
name = 'bpi_2012'
#name = 'bpi_2013'
#name = 'Road_Traffic_Fine_Management_Process'  

parser = {
    'train': True,
    'test': True,
    'model_class': 'AE',
    'model_name': '',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.5,
    'input_dir': '../input/{}/'.format(name), 
    'output_dir': './output/{}/'.format(name),
    'batch_size' : 16,
    'epochs' : 15,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 1000,
    'layer2': 100,
    'lr': 0.002,
    'betas': (0.9, 0.999),   
    'lr_decay': 0.90,
}

args = argparse.Namespace(**parser)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

## Train

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

## Validate and test

In [12]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [13]:
missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

In [14]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [15]:
missing_true_val.shape, missing_true_test.shape

((55671, 5), (47823, 5))

In [16]:
nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

In [17]:
nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

In [18]:
pd.isnull(normalized_missing_df).sum()

CaseID                    0
Activity             130920
CompleteTimestamp    131280
CumTimeInterval      131280
NormalizedTime       131280
dtype: int64

In [19]:
pd.isnull(missing_true_val).sum()

CaseID                   0
Activity             27799
CompleteTimestamp    27961
CumTimeInterval      27961
NormalizedTime       27961
dtype: int64

In [20]:
pd.isnull(missing_true_test).sum()

CaseID                   0
Activity             23817
CompleteTimestamp    23990
CumTimeInterval      23990
NormalizedTime       23990
dtype: int64

# Build model

## Define model

In [21]:
if args.model_class == 'AE':
    model = AE(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2)

if args.cuda:
    model.cuda()

## Define loss

In [22]:
# Define loss

def loss_function(recon_x, x, avai_mask):
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False) 
    return BCE

In [23]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [24]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam
lambda1 = lambda epoch: args.lr_decay ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

## Utils

In [25]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [26]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

In [27]:
def val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, nan_matrix_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    c_val = complete_matrix_w_normalized_time_val
    c_val = Variable(torch.Tensor(c_val).float())
    
    nan_matrix_val = Variable(torch.Tensor(nan_matrix_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        nan_matrix_val = nan_matrix_val.cuda()
        
    recon_data = model(m_val)
        
    loss = loss_function(recon_data, c_val, nan_matrix_val)
    return loss.data[0]/missing_matrix_w_normalized_time_val.shape[0]

```python
def val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
       pad_matrix_val, cols_w_normalized_time, val_row_num,
       nan_time_index_val, nan_activity_index_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        
    recon_val = model(m_val)
    
    recon_df_w_normalized_time = convert2df(recon_val, pad_matrix_val, cols_w_normalized_time, val_row_num)
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_val, min_max_storage)
    submission_df = getSubmission(recon_df_w_time, missing_true_val, complete_true_val, first_timestamp)
    
    #evaluate
    time, acc = evaluation(submission_df, nan_time_index_val, nan_activity_index_val)
    
    
    return time/86400+1/acc
```

In [28]:
missing_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,A_SUBMITTED-COMPLETE,NaN,NaN,NaN
1,7853,NaN,2012-01-05 06:57:15.858,8291911.312,0.0
2,7853,A_DECLINED-COMPLETE,NaN,NaN,NaN
3,7854,A_SUBMITTED-COMPLETE,2012-01-05 06:59:14.543,8292029.997,0.0
4,7854,NaN,NaN,NaN,NaN


In [29]:
complete_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,7853,A_SUBMITTED-COMPLETE,2012-01-05 06:57:15.554,8291911.008,0.000000e+00
1,7853,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:57:15.858,8291911.312,7.624972e-03
2,7853,A_DECLINED-COMPLETE,2012-01-05 06:57:55.323,8291950.777,9.974918e-01
3,7854,A_SUBMITTED-COMPLETE,2012-01-05 06:59:14.543,8292029.997,0.000000e+00
4,7854,A_PARTLYSUBMITTED-COMPLETE,2012-01-05 06:59:15.103,8292030.557,5.115717e-07


# Train model

In [30]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader, 
                                                     complete_matrix_w_normalized_time_trainLoader,
                                                     avai_matrix_trainLoader)):
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())

        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()

            
        optimizer.zero_grad()
        
        recon_data = model(m_data)
        
        loss = loss_function(recon_data, c_data, avai_mask)
        
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [31]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        #scheduler.step()
        train_loss = train(epoch, model, optimizer)
        #val_score = val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
        #                    pad_matrix_val, cols_w_normalized_time, val_row_num, 
        #                    nan_time_index_val, nan_activity_index_val)
        val_score = val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, avai_matrix_val)
        
        if epoch == 1:
            current_best = val_score
            save_model(model, epoch, val_score)
        
        else:
            if val_score < current_best:
                current_best = val_score
                save_model(model, epoch, val_score)
                
        scheduler.step(val_score)        
        print('====> Epoch {} | Train loss: {:.4f} | Val score: {:.4f}'.format(
                          epoch, train_loss, val_score))
else:
    load_model(model, args.model_name)

====> Epoch 1 | Train loss: 29.7017 | Val score: 22.6516
====> Epoch 2 | Train loss: 15.7714 | Val score: 18.3483
====> Epoch 3 | Train loss: 13.5143 | Val score: 16.9332
====> Epoch 4 | Train loss: 11.6584 | Val score: 15.9344
====> Epoch 5 | Train loss: 10.0436 | Val score: 15.2593
====> Epoch 6 | Train loss: 8.6513 | Val score: 14.9051
====> Epoch 7 | Train loss: 7.4855 | Val score: 14.7402
====> Epoch 8 | Train loss: 6.5661 | Val score: 14.7209
====> Epoch 9 | Train loss: 5.8377 | Val score: 14.9993
====> Epoch 10 | Train loss: 5.2171 | Val score: 15.4431
====> Epoch 11 | Train loss: 4.7275 | Val score: 14.9437
====> Epoch 12 | Train loss: 4.3773 | Val score: 15.0849
====> Epoch 13 | Train loss: 4.0704 | Val score: 15.8734
====> Epoch 14 | Train loss: 3.7599 | Val score: 15.7853
====> Epoch 15 | Train loss: 3.5408 | Val score: 15.7952


# Predict and evaluate

In [32]:
if args.test:
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    submission = fixTime(submission_df)
    
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test,show=True)
    print('\n')
    
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
Transforming Normalized Time to Time...
Getting submission...


/home/pllab/miniconda2/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Testing...
Number of missing Time: 23990
Mean Absolute Error: 1.0598 day(s)
Root Mean Squared Error: 2.4559 day(s)
Number of missing Activity: 23817
Accuracy: 74.90%


Saving submission...
Done!


In [ ]:
submission_df.head(10)

In [ ]:
submission.head(10)

In [33]:
#bpi2013
#7.6
#75%

In [34]:
#bpi2012
#0.59
#75%

In [35]:
#helpdesk
#3.9
#85